In [1]:
import pyspark as ps
from pyspark import SparkConf, SparkContext
from __future__ import unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
import gzip
import spacy
%matplotlib inline
np.random.seed(32113)
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF
from nltk.corpus import stopwords
from spacy.en import English
from sklearn.grid_search import GridSearchCV
import string
import data_prep_for_test_run as dp 
parser = English()
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import isnan, when, count, col
import pyspark.sql.functions as pys_fun
import math

/Users/keisukeirie/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/keisukeirie/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
hc = ps.HiveContext(sc)
sql = ps.SQLContext(sc)

In [3]:
df = spark.read.csv('sparkprocess1.csv', header=True, inferSchema=True)

In [4]:
print((df.count(), len(df.columns)))

(4234, 111)


#### NOTES:
I notice something. I realize that there are 25 samples where reviewText is null(!?).  
I was not able to run TFIDF for this reason (or collect tokenized data)...  

In [5]:
df.filter(df.reviewText.isNotNull()).count()

4209

In [6]:
df.filter(df.reviewText.isNull()).count()

25

In [7]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+-------+-------------------+--------------------+------------------+-----------+----+-----------+------------+-----------------+-------------+-------------+-------------+----------------+---------+--------+---------------+---+------------------+------------+--------+----+------------------+-------------------+------------------+-----------------------+----------+-----------+----------+-----------+------------+-----------+-----------+-----+---------------+-------------+-----------+----------+----------+--------+--------+-------------+------------+--------+-------------------+---+--------+--------------+------------+-----+------------------------+----+-------+----------+--------+--------+---------+-------+------------+--------+--------------------+------------------+--------------+----+--------------+-------+--------+----------+----------+---------+--------------------+-----------------+-------+------------+-----------+-----+--------+----------+--------------+------------+---

In [8]:
df = df.na.drop(subset=["reviewText"])

In [9]:
df.filter(df.reviewText.isNull()).count()

0

In [10]:
df.schema.names

['reviewText',
 'overall',
 'helpful_total_votes',
 'num_of_helpful_votes',
 'helpful_percentage',
 'Text_length',
 'asin',
 'rank_values',
 'num_category',
 'Screen Protectors',
 'PlayStation 4',
 'PlayStation 3',
 'PlayStation 2',
 'Game Boy Advance',
 'Joysticks',
 'GameCube',
 'Commodore Amiga',
 '3DO',
 'Cases & Protectors',
 'More Systems',
 'Sony PSP',
 'LIVE',
 'Sega Master System',
 'Fitness Accessories',
 'Subscription Cards',
 'Points & Currency Cards',
 'Atari 2600',
 'Controllers',
 'Light Guns',
 'Nintendo 64',
 'Nintendo 3DS',
 'Linux Games',
 'Accessories',
 'Skins',
 'Steering Wheels',
 'Racing Wheels',
 'PlayStation',
 'Networking',
 'Atari 7800',
 'Xbox 360',
 'Chargers',
 'Kids & Family',
 'ColecoVision',
 'Gamepads',
 'Digital Games & DLC',
 'PC',
 'Consoles',
 'Game Boy Color',
 'Nintendo NES',
 'Drums',
 'MMO & Free-to-Play Games',
 'Xbox',
 'Sega CD',
 'Faceplates',
 'Game Boy',
 'Adapters',
 'Batteries',
 'Fire TV',
 'Sega Genesis',
 'Speakers',
 'Batteries & C

In [11]:
column222 = [u'reviewerID', u'asin', u'reviewerName', u'reviewText', u'overall',
       u'summary', u'helpful_total_review', u'num_of_helpful_review',
       u'helpful_percent', u'text_length', u'price', u'rank_values',
       u'num category', u'Sony PSP', u'PlayStation', u'LIVE', u'PlayStation 3',
       u'PlayStation 2', u'Game Boy Advance', u'Xbox 360', u'Joysticks',
       u'GameCube', u'PC Game Downloads', u'Chargers', u'Kids & Family',
       u'Remotes', u'Memory', u'Gamepads', u'Networking',
       u'Digital Games & DLC', u'Nintendo DS', u'Cases & Protectors',
       u'More Systems', u'PlayStation Vita', u'Adapters', u'Digital Games',
       u'Mac', u'PC', u'Consoles', u'Game Boy Color', u'Dance Mats',
       u'Nintendo NES', u'Drums', u'Interactive Gaming Figures', u'Xbox One',
       u'Screen Protectors', u'Mac Game Downloads', u'Downloadable Content',
       u'Fitness Accessories', u'MMO & Free-to-Play Games',
       u'Subscription Cards', u'Points & Currency Cards', u'Flight Controls',
       u'Currency Cards', u'Xbox', u'Controllers', u'Cables & Adapters',
       u'Games', u'Batteries & Chargers', u'Game Boy', u'Light Guns',
       u'Nintendo 64', u'PlayStation 4', u'Super Nintendo', u'Guitars',
       u'Wii U', u'Nintendo 3DS', u'Steering Wheels', u'Headsets',
       u'Accessories', u'Wii', u'Skins', u'Hardware', u'Linux Games',
       u'Batteries', u'Gaming Mice', u'Sega Genesis', u'Accessory Kits',
       u'Fire TV', u'Cables', u'Gaming Keyboards', u'Casual Games',
       u'Racing Wheels', u'Software', u'Video Games']

In [12]:
[co for co in df.schema.names if co not in column222]

['helpful_total_votes',
 'num_of_helpful_votes',
 'helpful_percentage',
 'Text_length',
 'num_category',
 'Commodore Amiga',
 '3DO',
 'Sega Master System',
 'Atari 2600',
 'Atari 7800',
 'ColecoVision',
 'Sega CD',
 'Faceplates',
 'Speakers',
 'Sega Game Gear',
 'PDAs',
 'Sega Dreamcast',
 'Atari 5200',
 'Keyboards',
 'Commodore 64',
 'Microphones',
 'Atari Lynx',
 'Intellivision',
 'NEOGEO Pocket',
 'TurboGrafx 16',
 'Sensor Bars',
 'Atari Jaguar',
 'Sega Saturn',
 'id',
 'software',
 'Video_Games',
 'below20',
 'below50',
 'below100',
 'below300',
 'price_unknown']

## Tokenizer and Stopword in Spark
Here, I am applying Tokenizer and stopword to my review contents.  
I realize that Spark tokenizer does not lemmatize/tokenize word accurately unlike SpaCy that I used previously.  
So I played around with it and decided to do following process:
1. tokenize review text.  
2. use stopwordremover to remove custom stopword I created with NLTK, SKlearn Stopwords.  
3. run stopwordremover 1 more time. This time, run it with the spark stopword.  
  
I also realize that running tokenizer does not take good care of special symbols (!, ., ?, etc) so I repeated the same procedure with RegexTokenizer with Pattern set as \\W+  
this should get rid of all special character that is in review contents.  

In [13]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.ml.feature import StopWordsRemover

In [14]:
#ran both tokenizer and regextokenizer
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
tokenized = tokenizer.transform(df)

#the regextokenizer here takes out any non-word character from a token.
#for instance, Spoiler! --> spoiler
regtokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W+")
reg = regtokenizer.transform(df)

In [15]:
tokenized.select('words').show()

+--------------------+
|               words|
+--------------------+
|[this, does, not,...|
|[my, son, has, be...|
|[unfortunately, i...|
|[animal, crossing...|
|[got, this, game,...|
|[i'm, going, to, ...|
|[the, all, new, s...|
|[the, starter, bu...|
|[atgames, is, a, ...|
|[i've, had, grand...|
|[**update:, many,...|
|[no, spoilers!, r...|
|[as, you'll, prob...|
|[i've, been, an, ...|
|[when, call, of, ...|
|[it's, amazing, w...|
|[imagine, you, ha...|
|[in, 2005,, f.e.a...|
|[while, the, past...|
|[we, got, our, ha...|
+--------------------+
only showing top 20 rows



In [16]:
#my STOPLIST from the original project. mix NLTK and SKlearn english stopwords.
STOPLIST = list(set(stopwords.words('english') + ["n't", "'s", "'m", "ca"] + \
                list(ENGLISH_STOP_WORDS))) +\
                 " ".join(string.punctuation).split(" ") + \
                 ["-----", "---", "...", "..", "....", "", " ", "\n", "\n\n"]

In [17]:
stopwordremove = StopWordsRemover(inputCol="words", outputCol="tokenized_filt" ,stopWords=STOPLIST)

In [18]:
#applying customized stopwordlist to tokenized words
tokenized2 = stopwordremove.transform(tokenized)
reg2 = stopwordremove.transform(reg)

In [19]:
tokenized2.select("tokenized_filt").show()

+--------------------+
|      tokenized_filt|
+--------------------+
|[come, usb, hook,...|
|[son, eagerly, an...|
|[unfortunately, c...|
|[animal, crossing...|
|[got, game, day, ...|
|[i'm, going, pref...|
|[new, slimmer, ps...|
|[starter, bundle,...|
|[atgames, chinese...|
|[i've, grand, sla...|
|[**update:, revie...|
|[spoilers!, read,...|
|[you'll, probably...|
|[i've, xbox, 360,...|
|[duty, arrived, p...|
|[it's, amazing, s...|
|[imagine, wonderf...|
|[2005,, f.e.a.r.,...|
|[past, year,, we'...|
|[got, hands, xbox...|
+--------------------+
only showing top 20 rows



you see that custom stopword is not working for words like i'm,it's,i've.  
So I decided to run another stopwordremover.  
This time, I am applying spark stopword by not specifying any custom stopwordlist.  

In [20]:
remove = StopWordsRemover(inputCol="tokenized_filt", outputCol="tokenized")
tokenized3 = remove.transform(tokenized2)
reg3 = remove.transform(reg2)

In [21]:
#Comparison between normal tokenizer and regex tokenizer results.
tokenized3.select('tokenized').show()
reg3.select('tokenized').show()

+--------------------+
|           tokenized|
+--------------------+
|[come, usb, hook,...|
|[son, eagerly, an...|
|[unfortunately, r...|
|[animal, crossing...|
|[got, game, day, ...|
|[going, preface, ...|
|[new, slimmer, ps...|
|[starter, bundle,...|
|[atgames, chinese...|
|[grand, slam, ten...|
|[**update:, revie...|
|[spoilers!, read,...|
|[probably, know,,...|
|[xbox, 360, owner...|
|[duty, arrived, p...|
|[amazing, sony, t...|
|[imagine, wonderf...|
|[2005,, f.e.a.r.,...|
|[past, year,, rea...|
|[got, hands, xbox...|
+--------------------+
only showing top 20 rows

+--------------------+
|           tokenized|
+--------------------+
|[come, usb, hook,...|
|[son, eagerly, an...|
|[unfortunately, r...|
|[animal, crossing...|
|[got, game, day, ...|
|[going, preface, ...|
|[new, slimmer, ps...|
|[starter, bundle,...|
|[atgames, chinese...|
|[grand, slam, ten...|
|[update, reviewer...|
|[spoilers, read, ...|
|[probably, know, ...|
|[xbox, 360, owner...|
|[duty, arrived, p...|
|[amazin

In [22]:
reg3=reg3.drop('words','tokenized_filt')

### Tokenizer result
I would say that Regextokenizer is doing it's job the way I wanted.  
Although the process of making tokenizer is super fast and easy, I wish lemmatizer and tokenizer were smarter like the one in SpaCy.  
But again, I can not complain about the speed of the process :-)

## TFIDF MATRIX


In [23]:
from pyspark.ml.feature import HashingTF, IDF

In [24]:
hashingTF = HashingTF(inputCol="tokenized", outputCol="TTT", numFeatures=10000)
featurizedData = hashingTF.transform(reg3)

In [25]:
featurizedData.select('TTT').take(1)

[Row(TTT=SparseVector(10000, {1: 1.0, 253: 1.0, 404: 4.0, 677: 1.0, 721: 1.0, 1020: 1.0, 1274: 1.0, 1561: 1.0, 1742: 1.0, 1821: 1.0, 1964: 2.0, 2133: 2.0, 2157: 1.0, 2188: 1.0, 2587: 1.0, 2722: 1.0, 2766: 1.0, 2801: 1.0, 3051: 1.0, 3516: 1.0, 3525: 1.0, 3765: 1.0, 3913: 1.0, 4040: 1.0, 4099: 1.0, 4127: 1.0, 4227: 2.0, 4260: 2.0, 4467: 1.0, 4489: 1.0, 5015: 1.0, 5086: 2.0, 5089: 1.0, 5141: 1.0, 5483: 1.0, 5666: 2.0, 6252: 1.0, 6364: 1.0, 6537: 1.0, 6581: 1.0, 6692: 3.0, 7040: 1.0, 7477: 1.0, 7527: 2.0, 7550: 1.0, 7610: 2.0, 7705: 1.0, 7712: 1.0, 7779: 1.0, 7788: 1.0, 8117: 1.0, 8151: 2.0, 8567: 1.0, 8831: 1.0, 9161: 1.0, 9451: 1.0, 9527: 1.0, 9567: 1.0, 9623: 1.0, 9845: 1.0, 9977: 2.0}))]

In [26]:
idf = IDF(minDocFreq=2, inputCol="TTT", outputCol="TFIDF_features")
idfModel = idf.fit(featurizedData)
TFIDF_model = idfModel.transform(featurizedData)

In [355]:
TFIDF_model.select('TFIDF_features').take(1)

[Row(TFIDF_features=SparseVector(10000, {1: 2.4077, 253: 2.2814, 404: 13.2331, 677: 3.1303, 721: 1.5263, 1020: 4.2509, 1274: 4.7076, 1561: 2.8646, 1742: 2.1011, 1821: 1.5836, 1964: 3.046, 2133: 3.9644, 2157: 3.8454, 2188: 4.2181, 2587: 4.2181, 2722: 3.8344, 2766: 4.0967, 2801: 1.9839, 3051: 1.8202, 3516: 4.7076, 3525: 1.0063, 3765: 2.7212, 3913: 3.7105, 4040: 5.7062, 4099: 1.9483, 4127: 3.8566, 4227: 6.7929, 4260: 2.0362, 4467: 3.4549, 4489: 1.1883, 5015: 5.9473, 5086: 7.977, 5089: 1.9483, 5141: 2.4901, 5483: 3.9264, 5666: 3.8932, 6252: 3.032, 6364: 1.9416, 6537: 1.3735, 6581: 3.6267, 6692: 12.2902, 7040: 2.3972, 7477: 4.474, 7527: 5.8582, 7550: 2.6151, 7610: 9.3633, 7705: 6.148, 7712: 1.0563, 7779: 1.2728, 7788: 3.2332, 8117: 3.2454, 8151: 4.415, 8567: 2.4263, 8831: 2.9336, 9161: 3.3548, 9451: 3.2036, 9527: 1.4659, 9567: 3.4326, 9623: 5.8603, 9845: 3.6724, 9977: 8.6397}))]

In [444]:
TFIDF_model = TFIDF_model.withColumn("id2", monotonically_increasing_id())

In [447]:
rdd_id = TFIDF_model.rdd.map(lambda x: [x.id2]*len(x.TFIDF_features.indices)).flatMap(lambda x: x)

In [448]:
rdd_indices = TFIDF_model.rdd.map(lambda x: x.TFIDF_features.indices).flatMap(lambda x: x)

In [449]:
rdd_tfidf = TFIDF_model.rdd.map(lambda x: x.TFIDF_features.values).flatMap(lambda x: x)

In [450]:
rd_zip1 = rdd_id.zip(rdd_indices)
rdd_zipped = rd_zip1.zip(rdd_tfidf).map(lambda x: (int(x[0][0]),int(x[0][1]),float(x[1])))


In [452]:
rdd_zipped.take(2)

[(0, 1, 2.4076817215940007), (0, 253, 2.2814327179888187)]

In [484]:
test = hc.createDataFrame(rdd_zipped)
test = test.select(col('_1').alias("USER_ID").cast(IntegerType()), col('_2').alias("tfidf_index").cast(IntegerType()), col('_3').alias("tfidf_term"))
test = test.withColumn("tfidf_ROUND", pys_fun.format_number(test.tfidf_term, 3).cast(FloatType()))
test = test.drop('tfidf_term')

In [485]:
test.printSchema()

root
 |-- USER_ID: integer (nullable = true)
 |-- tfidf_index: integer (nullable = true)
 |-- tfidf_ROUND: float (nullable = true)



In [466]:
print(rdd_id.count())
print(rdd_indices.count())
print(rdd_tfidf.count())

605816
605816
605816


In [486]:
als = ALS(rank=8, maxIter=5, userCol="USER_ID", itemCol="tfidf_index", ratingCol="tfidf_ROUND", nonnegative=True)

In [489]:
als.fit(test)

Py4JJavaError: An error occurred while calling o7440.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 309.0 failed 1 times, most recent failure: Lost task 0.0 in stage 309.0 (TID 770, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.evalExpr1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.next(SerDeUtil.scala:121)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.next(SerD

Rating matrix:
(USER_ID, list index of NON-Zero value, non-zero value)

USER_ID = rdd_tfidf[0][0]
index = rdd_tfidf[0][1]
values = rdd)tfidf[0][2]

In [255]:
rdd_tfidf2=rdd_tfidf.map(lambda x: (x[0][0]*len(x[0][1]),x[0][1],x[0][2]))

In [276]:
df_mat = hc.createDataFrame(rdd_tfidf)

TypeError: not supported type: <type 'numpy.int32'>

In [128]:
df_mat.show()

+-------------+--------------------+
|           id|      TFIDF_features|
+-------------+--------------------+
|  25769803779|(10000,[1,253,404...|
| 154618822659|(10000,[76,103,11...|
| 274877907016|(10000,[69,137,20...|
| 292057776131|(10000,[68,78,149...|
| 326417514596|(10000,[1820,1997...|
| 446676598802|(10000,[230,246,2...|
| 472446402575|(10000,[68,172,22...|
| 481036337164|(10000,[8,32,68,1...|
| 592705486860|(10000,[18,77,78,...|
| 627065225225|(10000,[148,229,2...|
| 730144440325|(10000,[8,68,147,...|
| 738734374913|(10000,[25,26,43,...|
| 816043786244|(10000,[224,545,5...|
| 824633720837|(10000,[71,103,13...|
| 841813590032|(10000,[23,63,78,...|
| 927712935944|(10000,[103,139,2...|
| 927712935972|(10000,[185,711,7...|
| 953482739720|(10000,[8,45,68,7...|
|1013612281875|(10000,[57,127,12...|
|1099511627788|(10000,[36,73,185...|
+-------------+--------------------+
only showing top 20 rows



In [144]:
from pyspark.mllib.linalg import SparseVector, DenseVector

In [202]:
rdd_tfidf = df_mat.rdd.map(lambda x: (x.TFIDF_features).toArray())

In [132]:
als = ALS(rank=10, maxIter=5, seed=0)
model = als.fit(df_mat)

IllegalArgumentException: u'Field "user" does not exist.'

# from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = sqlContext.createDataFrame([
    (0, "Hi I heard about Spark"),
    (0, "I wish Java could use case classes"),
    (1, "Logistic regression models are neat")
], ["label", "sentence"])
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
for features_label in rescaledData.select("features", "label").take(3):
    print(features_label)

In [91]:
featurizedData.select("rawFeatures").collect()

[Row(rawFeatures=SparseVector(20, {0: 1.0, 5: 1.0, 9: 1.0, 17: 2.0})),
 Row(rawFeatures=SparseVector(20, {2: 1.0, 7: 1.0, 9: 3.0, 13: 1.0, 15: 1.0})),
 Row(rawFeatures=SparseVector(20, {4: 1.0, 6: 1.0, 13: 1.0, 15: 1.0, 18: 1.0}))]